# 📊 Análise de Aulas Coletadas

Este notebook tem como objetivo analisar os dados extraídos pelo scraper do portal da Seduc. Vamos verificar a quantidade de aulas por disciplina, por turma e a distribuição ao longo do tempo para identificar possíveis falhas na coleta e entender o panorama geral dos registros.

## 1. Carregamento e Preparação dos Dados

Primeiro, vamos importar as bibliotecas necessárias e carregar o arquivo `aulas_coletadas.json` em um DataFrame do Pandas, que é a estrutura ideal para análise de dados em Python.

In [ ]:
import json
import pandas as pd
import plotly.express as px
import os

# Define o estilo dos gráficos
px.defaults.template = "plotly_dark"

# Constrói o caminho para o arquivo de dados
file_path = os.path.join('..', 'data', 'aulas_coletadas.json')

# Carrega os dados
try:
    df = pd.read_json(file_path)
    print(f"Arquivo '{file_path}' carregado com sucesso!")
    print(f"Total de registros encontrados: {len(df)}")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{file_path}' não foi encontrado. Execute o scraper primeiro.")
    df = pd.DataFrame() # Cria um DataFrame vazio para evitar erros posteriores

# Exibe as primeiras linhas e informações do DataFrame
if not df.empty:
    display(df.head())
    print("\nInformações do DataFrame:")
    df.info()

In [ ]:

import os

# --- Caminhos para os arquivos ---
# Define o caminho para a pasta 'data' e para o arquivo de saída.
# Esta abordagem com '..' sobe um nível no diretório, sendo mais flexível.
# Se você executar este código da raiz do projeto, pode usar diretamente 'data/aulas_coletadas.json'.
json_input_path = os.path.join('data', 'aulas_coletadas.json')
csv_output_path = 'aulas_profhelio2025.csv' # Salva o CSV na pasta atual

# 1. Carregar os dados do arquivo JSON para um DataFrame
try:
    # pd.read_json é a função correta para ler um arquivo JSON
    df = pd.read_json(json_input_path)
    print(f"Arquivo '{json_input_path}' carregado com sucesso.")
except FileNotFoundError:
    print(f"ERRO: Arquivo de entrada não encontrado em '{json_input_path}'. Verifique se o caminho está correto.")
    exit()
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo JSON: {e}")
    exit()

# 2. Preparar o DataFrame para exportação (se necessário)
# Adiciona colunas que não existem no JSON, como 'Ações'
df['Ações'] = 'Visualizar | Editar' 

# Seleciona e reordena as colunas na ordem desejada para o CSV
# As chaves aqui devem corresponder exatamente às chaves do seu arquivo JSON
df_export = df[[
    'data_cadastro',
    'horario',
    'dataAula',
    'turma',
    'componenteCurricular',
    'status',
    'Ações'
]]

# 3. Defina a lista com os cabeçalhos exatos que você deseja no arquivo CSV
cabecalho_desejado = [
    'data_cadastro',
    'Horário_(inicial_~_final)',
    'Data Aula',
    'Turma',
    'Componente',
    'Situação',
    'Ações'
]

# 4. Salve o DataFrame em CSV com os parâmetros corretos
df_export.to_csv(
    csv_output_path,
    header=cabecalho_desejado, # Usa sua lista de cabeçalhos personalizados
    index=False,               # Essencial para não salvar o índice do DataFrame
    sep=';',                   # Ponto e vírgula é mais compatível com o Excel no Brasil
    encoding='utf-8-sig'       # Garante a correta exibição de acentos e caracteres especiais
)

print(f"\nDataFrame salvo com sucesso em '{csv_output_path}'")

### Limpeza e Conversão de Tipos

A coluna `dataAula` está como texto. Vamos convertê-la para o tipo `datetime` para que possamos fazer análises temporais, como agrupar por mês ou semana.

In [ ]:
# Filtra o DataFrame para manter apenas as linhas onde o status é 'Aula confirmada'
df_confirmadas = df[df['status'] == 'Aula confirmada'].copy()

# Agora você pode continuar sua análise usando o df_confirmadas
print(f"Total de aulas: {len(df)}")
print(f"Total de aulas confirmadas: {len(df_confirmadas)}")

# Exemplo de como usar o novo DataFrame
display(df_confirmadas.head())

In [ ]:
# Salvando em CSV da forma recomendada
df.to_csv(
    'clientes.csv',      # Nome do arquivo
    index=False,         # Não salvar o índice
    sep=';',             # Usar ponto e vírgula como separador
    encoding='utf-8-sig' # Garantir compatibilidade de caracteres e com Excel
)

In [ ]:
if not df.empty:
    # Converte a coluna 'dataAula' para o formato de data, tratando possíveis erros
    df['dataAula'] = pd.to_datetime(df['dataAula'], format='%d/%m/%Y', errors='coerce')

    # Remove linhas onde a conversão de data falhou (se houver)
    df.dropna(subset=['dataAula'], inplace=True)

    # Extrai o mês e o ano para facilitar o agrupamento
    df['mes'] = df['dataAula'].dt.strftime('%Y-%m')

    print("Coluna 'dataAula' convertida para datetime.")
    display(df.head())

## 2. Análise Estatística

Agora vamos começar a agregar os dados para obter insights.

### Contagem de Aulas por Disciplina

Isso nos mostrará o total de registros coletados para cada componente curricular. Se uma disciplina tem um número muito baixo de aulas, pode ser um indicativo de que a coleta falhou para ela em algum momento.

In [ ]:
if not df.empty:
    # Agrupa por componente curricular e conta o número de aulas
    aulas_por_disciplina = df['componenteCurricular'].value_counts().reset_index()
    aulas_por_disciplina.columns = ['Disciplina', 'Quantidade de Aulas']

    print("Total de aulas coletadas por disciplina:")
    display(aulas_por_disciplina)

In [ ]:
if not df.empty:
    fig = px.bar(
        aulas_por_disciplina, 
        x='Disciplina', 
        y='Quantidade de Aulas', 
        title='Quantidade de Aulas Registradas por Disciplina',
        text='Quantidade de Aulas',
        color='Disciplina'
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

### Contagem de Aulas por Turma

Similarmente, vamos verificar a distribuição de aulas por turma.

In [ ]:
if not df.empty:
    aulas_por_turma = df['turma'].value_counts().reset_index()
    aulas_por_turma.columns = ['Turma', 'Quantidade de Aulas']

    print("Total de aulas coletadas por turma:")
    display(aulas_por_turma)

In [ ]:
if not df.empty:
    fig = px.pie(
        aulas_por_turma, 
        names='Turma', 
        values='Quantidade de Aulas', 
        title='Distribuição de Aulas por Turma',
        hole=0.3
    )
    fig.update_traces(textinfo='percent+label', pull=[0.05] * len(aulas_por_turma))
    fig.show()

### Distribuição de Aulas ao Longo do Tempo

Um histograma por data nos ajuda a ver se há meses ou períodos específicos com poucas ou nenhuma aula registrada, o que pode indicar problemas na paginação ou interrupção do scraper durante a coleta desses períodos.

In [ ]:
if not df.empty:
    aulas_por_mes = df.groupby('mes').size().reset_index(name='Quantidade de Aulas')

    fig = px.bar(
        aulas_por_mes,
        x='mes',
        y='Quantidade de Aulas',
        title='Distribuição de Aulas Registradas por Mês',
        text='Quantidade de Aulas'
    )
    fig.update_traces(textposition='outside')
    fig.update_xaxes(type='category') # Trata os meses como categorias para melhor visualização
    fig.show()

## 3. Análise Detalhada por Disciplina e Turma

Vamos cruzar as informações para ver quantas aulas de cada disciplina foram coletadas para cada turma.

In [ ]:
if not df.empty:
    # Cria uma tabela cruzada (crosstab) para ver a contagem de aulas por turma e disciplina
    tabela_cruzada = pd.crosstab(df['turma'], df['componenteCurricular'])

    print("Tabela Cruzada: Aulas por Turma e Disciplina")
    display(tabela_cruzada)

In [ ]:
if not df.empty:
    # Para visualizar melhor, podemos usar um mapa de calor
    fig = px.imshow(
        tabela_cruzada,
        text_auto=True, # Mostra os números dentro das células
        aspect="auto",
        title='Mapa de Calor: Aulas Coletadas por Turma e Disciplina',
        labels=dict(x="Disciplina", y="Turma", color="Quantidade")
    )
    fig.update_xaxes(side="top")
    fig.show()

## 4. Conclusões e Próximos Passos

Com base nos gráficos e tabelas acima, podemos tirar algumas conclusões:

1.  **Total de Aulas:** O número total coletado foi de **{len(df) if not df.empty else 0}**. Se a sua expectativa era de ~780, temos uma diferença a ser investigada.
2.  **Distribuição por Disciplina:** Verifique no primeiro gráfico de barras se alguma disciplina tem um número de aulas drasticamente menor que as outras. Isso pode indicar que o scraper foi interrompido ou que a paginação falhou especificamente para ela.
3.  **Distribuição por Mês:** O histograma mensal é crucial. Se você vir um mês com um número muito baixo de aulas (quando deveria ter muitas), é um forte indício de que a coleta para aquele período foi incompleta.
4.  **Mapa de Calor:** O mapa de calor nos dá a visão mais detalhada. Células com valor `0` ou valores muito baixos são os pontos exatos onde a coleta pode ter falhado.

**Sugestão de Investigação:**
Foque nas disciplinas e meses com os menores números. Tente rodar o scraper novamente, talvez filtrando para coletar apenas uma dessas turmas/disciplinas problemáticas, e observe o console para ver se ocorre algum erro de `Timeout` ou outro problema durante a paginação.